9.3 영어/한국어 Word2Vec 실습

1. 영어 Word2Vec 만들기

1) 훈련데이터 이해하기

In [1]:
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

In [2]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x2c64cfeb0d0>)

 xml 문법으로 작성되어 있어 전처리가 필요하다. <content>와 </content> 사이의 내용을 필요로 한다.

2) 훈련데이터 전처리

In [4]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower()) # a-z , 0-9 제와 나머지 공백처리인듯
     normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

In [5]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273380


In [6]:
# 샘플 3개만 출력
for line in result[:3]:
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


3) Word2Vec 훈련시키기

In [1]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.models import FastText

model = Word2Vec(sentences=result,  vector_size = 100, window = 5, min_count = 5, workers = 4, sg = 0)
model_f = FastText(result, vector_size=100, window=5, min_count=5, workers=4, sg=1)

NameError: name 'result' is not defined

size - 워드 벡터의 특징 값 즉, 임베딩 된 벡터의 차원\
window - 컨텍스트 위도우 크기\
min_count - 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)\
workers - 학습을 위한 프로세스 수\
sg - 0은 CBOW , 1은 Skip-gram

Word2Vec은 입력한 단어에 대해서 가장 유사한 단어들을 출력하는 model.wv.most_similar을 지원한다.

In [22]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.8688222765922546), ('guy', 0.8027599453926086), ('lady', 0.7912447452545166), ('boy', 0.7590741515159607), ('gentleman', 0.7496266961097717), ('soldier', 0.7435334920883179), ('girl', 0.730536162853241), ('kid', 0.6947352886199951), ('poet', 0.6563255190849304), ('surgeon', 0.6559193730354309)]


In [30]:
model_f.wv.most_similar("electrofishing")

[('electrolux', 0.867465078830719),
 ('electrolyte', 0.8635886907577515),
 ('electro', 0.8544803261756897),
 ('electroshock', 0.8432216644287109),
 ('electroencephalogram', 0.837772011756897),
 ('electrochemical', 0.8375294804573059),
 ('electrogram', 0.8312769532203674),
 ('airbus', 0.8268918395042419),
 ('electrons', 0.8255434632301331),
 ('electronic', 0.8226607441902161)]

4) Word2Vec 모델 저장하고 로드하기

In [31]:
model.wv.save_word2vec_format('eng_w2v') # 모델 저장
model_f.wv.save_word2vec_format('eng_w2v_f') # 모델 저장(fasttext)

loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드


In [10]:
# 로드한 모델에 대해서 다시 man과 유사한 단어를 출력
model_result = loaded_model.most_similar("man")
print(model_result)

[('woman', 0.8688222765922546), ('guy', 0.8027599453926086), ('lady', 0.7912447452545166), ('boy', 0.7590741515159607), ('gentleman', 0.7496266961097717), ('soldier', 0.7435334920883179), ('girl', 0.730536162853241), ('kid', 0.6947352886199951), ('poet', 0.6563255190849304), ('surgeon', 0.6559193730354309)]


2. 한국어 Word2Vec 만들기(by 네이버 영화 리뷰)

In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt
from tqdm import tqdm

In [12]:
# 네이버 영화 리뷰 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

('ratings.txt', <http.client.HTTPMessage at 0x2c62f8dd4b0>)

In [13]:
train_data = pd.read_table('ratings.txt')

In [14]:
train_data[:5] # 상위 5개 출력

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [15]:
print(len(train_data)) # 리뷰 갯수 출력

200000


In [16]:
# NULL 값 존재 유무
print(train_data.isnull().values.any())

True


In [17]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [18]:
# 결측값이 삭제 된 리뷰 데이터 수
print(len(train_data)) # 리뷰 개수 출력

199992


In [19]:
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

C:\Users\rladu\AppData\Local\Temp\ipykernel_10076\2876022004.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


In [20]:
train_data[:5] # 상위 5개 출력

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1
2,4655635,폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고,1
3,9251303,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1


In [21]:
# 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# 형태소 분석기 OKT를 사용한 토큰화 작업 (다소 시간 소요, 아마 데이터가 많아서 인듯)
okt = Okt()

tokenized_data = []
for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data.append(stopwords_removed_sentence)

NameError: name 'tqdm' is not defined

In [ ]:
# 리뷰 길이 분포 확인
print('리뷰의 최대 길이 :',max(len(review) for review in tokenized_data))
print('리뷰의 평균 길이 :',sum(map(len, tokenized_data))/len(tokenized_data))
plt.hist([len(review) for review in tokenized_data], bins = 50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = tokenized_data, size = 100, window = 5, min_count = 5, workers = 4, sg = 0)

In [ ]:
# 완성된 임베딩 매트릭스의 크기 확인
model.wv.vectors.shape

In [ ]:
print(model.wv.most_similar("최민식"))

In [ ]:
print(model.wv.most_similar("히어로"))

3. 사전 훈련된 Word2Vec 임베딩(Pre-trained Word2Vec embedding) 소개

보통 케라스의 Embedding()을 사용하여 갖고 있는 훈련 데이터로부터 훈련을 시키기도 하지만, 위키피디아 등의 방대한 데이터로 사전에 훈련된 워드 임베딩을 가지고 와서 해당 벡터들의 값을 원하는 작업에 사용 할 수도 있다.\
예를 들어 감성 분류 작업을 진행할때 훈련 데이터의 양이 부족한 상황이라면, 방대한 데이터를 Word2Vec이나 GloVe 등으로 사전에 학습시켜 놓은 임베딩 벡터들을 가지고 와서 모델의 입력으로 사용하는것이 좋은 성능을 보일때도 있다.\
이번에는 구글이 제공하는 사전 훈련된 Word2Vec 모델을 사용하는 방법에 대해서 알아보려고 한다.(사전 훈련된 3백만 개의 단어 벡터를 제공, 각 임베딩 벡터의 차원은 300)\
모델 다운로드 경로 : https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit

In [6]:
import gensim
import urllib.request

# 구글의 사전 훈련된 Word2Vec 모델을 로드.
urllib.request.urlretrieve("https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz",filename="GoogleNews-vectors-negative300.bin.gz")
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

HTTPError: HTTP Error 404: Not Found

In [ ]:
print(word2vec_model.vectors.shape)

In [ ]:
# 사전 훈련된 임베딩을 사용하여 두 단어의 유사도를 계산해본다.
print(word2vec_model.similarity('this', 'is'))
print(word2vec_model.similarity('post', 'book'))

In [ ]:
# 단어 'book'의 벡터
print(word2vec_model['book'])

Word2Vec은 자연어 처리에서 단어를 밀집 벡터로 만들어주는 단어 임베딩 방법론이지만, 최근에는 자연어 처리를 넘어서 \
추천 시스템에도 사용되고 있는 모델이다. 적당하게 데이터를 나열해주면 Word2Vec은 위치가 근접한 데이터를 유사도가 높은\
벡터를 만들어준다는 점에서 착안된 아이디어이다. ex) 구글에 'item2vec'을 검색